In [42]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pycrfsuite
import os
import numpy as np
import math

print(sklearn.__version__)

0.19.0


In [43]:
# Divide in train and test files [80:20] 

# Directory having content

doc_dir = 'Data/content'

train_file_list = []
test_file_list = []

for f in os.listdir(doc_dir):
    #Random Sampling
    if np.random.uniform(0,1)< 0.8:
        train_file_list.append(f)
    else:
        test_file_list.append(f)

print len(train_file_list)
print len(test_file_list)

1037
270


In [44]:
# Build Tagged dataset to be fed in nltk hmm tagger
# It is a list of tagged sentences. 
# training_data = [ [(word, tag), (word, tag).....]
# [(word, tag), (word, tag).....]
#]

# Directory having tags
tag_dir = 'Data/new_tags'

training_data = []

for f in train_file_list:
    training_sentences =[]
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    lines_in_word_file = []
    lines_in_tag_file = []
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            pairs_in_line = []
            length = min(len(word_in_file), len(tag_in_file))
            #Create the word_tag pair
            for j in xrange(length):
                pairs_in_line.append((word_in_file[j], tag_in_file[j]));
            training_sentences.append(pairs_in_line)
    if len(training_sentences) > 0:
        training_data.extend(training_sentences)

#print training_data[0]

In [45]:
def word2features(sent, i):
    N = len(sent)
    word = sent[i][0]
    count = 0
    for j in xrange(N):
        if sent[j][0] == word:
            count = count + 1
    score = math.log10((1+float(count)/N) * (float(1+N-i)/N))
    
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'word.score=%d' % score,
    ]
    if i > 0:
        word1 = sent[i-1][0]
        tag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
           # '-1:tag1=' + tag1,
        ])
    else:
        features.append('BOS')
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
        ])
    else:
        features.append('EOS')
        
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2tags(sent):
    return [tag for word, tag in sent]

def sent2words(sent):
    return [word for word, tag in sent] 

In [46]:
print training_data[0]
print sent2features(training_data[0])[0]
print sent2tags(training_data[0])[0]
print sent2words(training_data[0])[0]

[('Five', 'O'), ('people', 'O'), ('were', 'O'), ('killed', 'O'), ('and', 'O'), ('over', 'O'), ('50', 'O'), ('injured', 'O'), ('in', 'O'), ('three', 'O'), ('blasts', 'O'), ('set', 'O'), ('off', 'O'), ('by', 'O'), ('insurgent', 'O'), ('outfit', 'O'), ('ULFA', 'ORG_Others'), ('here', 'O'), ('today', 'O'), ('hours', 'O'), ('before', 'O'), ('Union', 'O'), ('Home', 'O'), ('Minister', 'O'), ('P', 'PER_Others'), ('Chidambaram', 'PER_Others'), ("'s", 'O'), ('visit', 'O'), ('to', 'O'), ('review', 'O'), ('law', 'O'), ('and', 'O'), ('order', 'O'), ('situation', 'O'), ('in', 'O'), ('the', 'O'), ('state', 'O'), ('rocked', 'O'), ('by', 'O'), ('deadly', 'O'), ('blasts', 'O'), ('that', 'O'), ('left', 'O'), ('88', 'O'), ('dead', 'O'), ('two', 'O'), ('months', 'O'), ('ago', 'O'), ('.', 'O'), ('.', 'O'), ('.', 'O'), ('Three', 'O'), ('people', 'O'), ('were', 'O'), ('killed', 'O'), ('and', 'O'), ('35', 'O'), ('injured', 'O'), (',', 'O'), ('including', 'O'), ('four', 'O'), ('women', 'O'), (',', 'O'), ('when'

In [47]:
x_train = [sent2features(s) for s in training_data]
y_train = [sent2tags(s) for s in training_data]

In [48]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(x_train, y_train):
    trainer.append(xseq, yseq)

In [49]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [50]:
trainer.params()

['feature.minfreq',
 'feature.possible_states',
 'feature.possible_transitions',
 'c1',
 'c2',
 'max_iterations',
 'num_memories',
 'epsilon',
 'period',
 'delta',
 'linesearch',
 'max_linesearch']

In [51]:
# Train and save the model
trainer.train('osint-bomb-blast.crfsuite.wordscore')

In [52]:
tagger = pycrfsuite.Tagger()
tagger.open('osint-bomb-blast.crfsuite.wordscore')

In [53]:
# Create test data
test_data = []
for f in test_file_list:
    test_sentences =[]
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    lines_in_word_file = []
    lines_in_tag_file = []
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            pairs_in_line = []
            length = min(len(word_in_file), len(tag_in_file))
            #Create the word_tag pair
            for j in xrange(length):
                pairs_in_line.append((word_in_file[j], tag_in_file[j]));
            test_sentences.append(pairs_in_line)
    if len(test_sentences) > 0:
        test_data.extend(test_sentences)

print test_data[0]

[('-LCB-', 'O'), ('mosimage', 'O'), ('-RCB-', 'O'), ('Three', 'O'), ('persons', 'O'), (',', 'O'), ('including', 'O'), ('an', 'O'), ('11-year-old', 'O'), ('boy', 'O'), (',', 'O'), ('were', 'O'), ('killed', 'O'), ('and', 'O'), ('seventeen', 'O'), ('injured', 'O'), ('in', 'O'), ('a', 'O'), ('blast', 'O'), ('that', 'O'), ('shook', 'O'), ('Mehrauli', 'LOC_Event'), ('area', 'O'), ('of', 'O'), ('south', 'LOC_Others'), ('Delhi', 'LOC_Others'), ('.', 'O'), ('Injured', 'O'), ('have', 'O'), ('been', 'O'), ('rushed', 'O'), ('to', 'O'), ('AIIMStrauma', 'O'), ('centre', 'O'), (',', 'O'), ('police', 'O'), ('said.Sources', 'O'), ('in', 'O'), ('the', 'O'), ('police', 'O'), ('say', 'O'), ('two', 'O'), ('people', 'O'), ('have', 'O'), ('been', 'O'), ('detained', 'O'), ('at', 'O'), ('the', 'O'), ('Delhi', 'LOC_Others'), ('IGI', 'O'), ('airport', 'O'), ('in', 'O'), ('connection', 'O'), ('with', 'O'), ('the', 'O'), ('blast', 'O'), ('.', 'O'), ('Ammonium', 'O'), ('nitrate', 'O'), ('and', 'O'), ('nails', 'O'),

In [54]:
example_sent = test_data[0]
print(' '.join(sent2words(example_sent)))

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2tags(example_sent)))

-LCB- mosimage -RCB- Three persons , including an 11-year-old boy , were killed and seventeen injured in a blast that shook Mehrauli area of south Delhi . Injured have been rushed to AIIMStrauma centre , police said.Sources in the police say two people have been detained at the Delhi IGI airport in connection with the blast . Ammonium nitrate and nails were reportedly used to make the bomb , similar to the ones used in the September 13 blasts.The terror attack , within a stone 's throw of Delhi 's landmark Qutub Minar , came exactly a fortnight after a series of bombings left 24 people dead in the national capital . ( AIIMS helpline : 91-11-26588500 , 26589900 , 26588700 ; Safdarjung Hospital helpline : 91-11-2619469 ) Two youths on a motorcycleleft a container in a black polythene bag outside an electronic shop and sped off . It began smoking and soon after there was a huge explosion , said Pritam , an eyewitness . The motorcycle allegedly used by the suspects has been recovered by th

In [55]:
def report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [56]:
x_test = [sent2features(s) for s in test_data]
y_test = [sent2tags(s) for s in test_data]

In [57]:
%%time
y_pred = [tagger.tag(xseq) for xseq in x_test]

CPU times: user 1.3 s, sys: 0 ns, total: 1.3 s
Wall time: 1.31 s


In [58]:
print(report(y_test, y_pred))

             precision    recall  f1-score   support

    Accused       0.00      0.00      0.00        57
      Event       0.00      0.00      0.00        68
LOC_Accused       0.22      0.01      0.02       212
  LOC_Assoc       0.00      0.00      0.00        68
  LOC_Event       0.51      0.42      0.46      1366
 LOC_Others       0.55      0.52      0.54      1956
 LOC_Victim       0.00      0.00      0.00        51
   Location       0.20      0.01      0.01       132
ORG_Accused       0.74      0.74      0.74       641
  ORG_Assoc       0.00      0.00      0.00        10
 ORG_Others       0.68      0.55      0.61      2335
 ORG_Victim       0.22      0.02      0.04        98
PER_Accused       0.53      0.39      0.45       828
  PER_Assoc       0.00      0.00      0.00        53
 PER_Others       0.76      0.68      0.72      2084
 PER_Victim       0.35      0.22      0.27       319
     Victim       0.00      0.00      0.00         6

avg / total       0.59      0.50      0.54  

In [59]:
num_test_data = len(y_test)
print len(y_pred)
print num_test_data

270
270


In [60]:
# Test
actual_tag_count = dict()
matched_tag_count = dict()
matched_tag_count['PER_Others'] = 0
actual_tag_count['PER_Others'] = 0
matched_tag_count['PER_Victim'] = 0
actual_tag_count['PER_Victim'] = 0
matched_tag_count['PER_Accused'] = 0
actual_tag_count['PER_Accused'] = 0
matched_tag_count['ORG_Victim'] = 0
actual_tag_count['ORG_Victim'] = 0
matched_tag_count['ORG_Accused'] = 0
actual_tag_count['ORG_Accused'] = 0
matched_tag_count['ORG_Others'] = 0
actual_tag_count['ORG_Others'] = 0
matched_tag_count['LOC_Accused'] = 0
actual_tag_count['LOC_Accused'] = 0
matched_tag_count['LOC_Others'] = 0
actual_tag_count['LOC_Others'] = 0
matched_tag_count['LOC_Event'] = 0
actual_tag_count['LOC_Event'] = 0
matched_tag_count['LOC_Victim'] = 0
actual_tag_count['LOC_Victim'] = 0

for i in xrange(num_test_data):
    y_test_i = y_test[i]
    y_pred_i = y_pred[i]
    test_tag_length = len(y_test[i])
    predicted_tag_length = len(y_pred[i])
    for j in xrange(test_tag_length):
        if y_test_i[j] in actual_tag_count.keys():
            actual_tag_count[y_test_i[j]] = actual_tag_count[y_test_i[j]] + 1
        
        if y_test_i[j] == y_pred_i[j]:
            if y_test_i[j] in matched_tag_count.keys():
                matched_tag_count[y_test_i[j]] = matched_tag_count[y_test_i[j]] + 1

In [61]:
print('======= Accuracy Class Wise =====================\n')

print('Class    Matched Total %')
print('--------------------------')
print ('PER_Others: '+str(matched_tag_count['PER_Others'])+ ' ' + str(actual_tag_count['PER_Others'])+ ' ' + str(matched_tag_count['PER_Others']*100/actual_tag_count['PER_Others'])+'%')
print ('PER_Victim: '+str(matched_tag_count['PER_Victim'])+ ' ' + str(actual_tag_count['PER_Victim'])+ ' '+str(matched_tag_count['PER_Victim']*100/actual_tag_count['PER_Victim'])+'%')
print ('PER_Accused: '+str(matched_tag_count['PER_Accused'])+ ' ' + str(actual_tag_count['PER_Accused'])+ ' '+str(matched_tag_count['PER_Accused']*100/actual_tag_count['PER_Accused'])+'%')
print ('ORG_Victim: '+str(matched_tag_count['ORG_Victim']) + ' '+ str(actual_tag_count['ORG_Victim'])+ ' '+str(matched_tag_count['ORG_Victim']*100/actual_tag_count['ORG_Victim'])+'%')
print ('ORG_Accused: '+str(matched_tag_count['ORG_Accused']) + ' '+ str(actual_tag_count['ORG_Accused'])+ ' '+str(matched_tag_count['ORG_Accused']*100/actual_tag_count['ORG_Accused'])+'%')
print ('ORG_Others: '+str(matched_tag_count['ORG_Others']) + ' '+ str(actual_tag_count['ORG_Others'])+ ' '+str(matched_tag_count['ORG_Others']*100/actual_tag_count['ORG_Others'])+'%')
print ('LOC_Accused: '+str(matched_tag_count['LOC_Accused'])+ ' ' + str(actual_tag_count['LOC_Accused'])+ ' '+str(matched_tag_count['LOC_Accused']*100/actual_tag_count['LOC_Accused'])+'%')
print ('LOC_Others: '+str(matched_tag_count['LOC_Others'])+ ' ' + str(actual_tag_count['LOC_Others'])+ ' '+str(matched_tag_count['LOC_Others']*100/actual_tag_count['LOC_Others'])+'%')
print ('LOC_Event: '+str(matched_tag_count['LOC_Event']) + ' '+ str(actual_tag_count['LOC_Event'])+ ' '+str(matched_tag_count['LOC_Event']*100/actual_tag_count['LOC_Event'])+'%')
print ('LOC_Victim: '+str(matched_tag_count['LOC_Victim']) + ' '+ str(actual_tag_count['LOC_Victim'])+ ' '+str(matched_tag_count['LOC_Victim']*100/actual_tag_count['LOC_Victim'])+'%')

======= Accuracy Class Wise =====================

Class    Matched Total %
--------------------------
PER_Others: 1419 2084 68%
PER_Victim: 71 319 22%
PER_Accused: 324 828 39%
ORG_Victim: 2 98 2%
ORG_Accused: 473 641 73%
ORG_Others: 1295 2335 55%
LOC_Accused: 2 212 0%
LOC_Others: 1023 1956 52%
LOC_Event: 571 1366 41%
LOC_Victim: 0 51 0%
